In [ ]:
import albumentations as A
import cv2
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import timm
import os
import random
from collections import defaultdict
from glob import glob
from tqdm import tqdm
import pandas as pd

In [ ]:
base_path = "/kaggle/input/datasets/iowiqo/lab1vegs/imgs/"

In [ ]:
# важно - зафиксировать все сиды
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
class_to_idx = {
  "Апельсины": 0,
  "Бананы": 1,
  "Груши": 2,
  "Кабачки": 3,
  "Капуста": 4,
  "Картофель": 5,
  "Киви": 6,
  "Лимон": 7,
  "Лук": 8,
  "Мандарины": 9,
  "Морковь": 10,
  "Огурцы": 11,
  "Томаты": 12,
  "Яблоки зелёные": 13,
  "Яблоки красные": 14
}

In [ ]:
class EMA:
    def __init__(self, model, decay=0.999):
        self.model = model
        self.decay = decay
        self.shadow = {}
        self.backup = {}
        self.register()

    def register(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()

    def update(self, model=None):
        if model is None:
            model = self.model
        for name, param in model.named_parameters():
            if param.requires_grad:
                new_average = (1.0 - self.decay) * param.data + self.decay * self.shadow[name]
                self.shadow[name] = new_average.clone()

    def apply_shadow(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                self.backup[name] = param.data
                param.data = self.shadow[name]

    def restore(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                param.data = self.backup[name]
        self.backup = {}
        
    @property
    def ema(self):
        return self.model

In [ ]:
class MyDataset(Dataset):
    def __init__(self, images_filepaths, name2label,
                 base_transform=None,
                 strong_transform=None,
                 weak_classes=None):
        """
        images_filepaths: список путей или кортежей (path, label)
        name2label: словарь имя класса -> индекс (для старых данных, если paths без меток)
        Если images_filepaths содержит кортежи (path, label), то name2label игнорируется.
        """
        self.images_filepaths = images_filepaths
        self.name2label = name2label
        self.base_transform = base_transform
        self.strong_transform = strong_transform
        self.weak_classes = weak_classes or []

    def __len__(self):
        return len(self.images_filepaths)

    def __getitem__(self, idx):
        # Если images_filepaths хранит кортежи (path, label)
        if isinstance(self.images_filepaths[idx], (tuple, list)):
            image_filepath, label = self.images_filepaths[idx]
        else:
            # Старый вариант: только путь, метка из name2label
            image_filepath = self.images_filepaths[idx]
            label = self.name2label[os.path.normpath(image_filepath).split(os.sep)[-3]]

        image = cv2.imdecode(np.fromfile(image_filepath, dtype=np.uint8), cv2.IMREAD_UNCHANGED)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Выбор подходящего pipeline
        if label in self.weak_classes:
            transform = self.strong_transform
        else:
            transform = self.base_transform

        if transform is not None:
            image = transform(image=image)['image']

        return image, label


def train_test_split_from_directory(root_path, folder2class, train_size=0.8):
    train, test = [], []
    class_to_images = defaultdict(list)

    for class_name in os.listdir(root_path):
        class_path = os.path.join(root_path, class_name)
        if not os.path.isdir(class_path):
            continue

        for subclass_name in os.listdir(class_path):
            subclass_path = os.path.join(class_path, subclass_name)
            if not os.path.isdir(subclass_path):
                continue

            # Глобальный класс = имя первой папки (class_name)
            class_name = folder2class.get(class_name, class_name)

            images = glob(os.path.join(subclass_path, '*.jpg')) + \
                     glob(os.path.join(subclass_path, '*.png')) + \
                     glob(os.path.join(subclass_path, '*.jpeg'))

            class_to_images[class_name].extend(images)

    # Равномерное разбиение по каждому классу
    for cls_name, images in class_to_images.items():
        random.shuffle(images)
        split_idx = int(train_size * len(images))
        train.extend(images[:split_idx])
        test.extend(images[split_idx:])

    return train, test


In [ ]:
from sklearn.model_selection import StratifiedKFold
from collections import Counter

# Собираем все изображения из папки train
all_images = []
all_labels = []

dataset_path = base_path + 'train'
for class_name in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_name)
    if not os.path.isdir(class_path):
        continue
    # Если есть подпапки (например, разные партии)
    for subclass_name in os.listdir(class_path):
        subclass_path = os.path.join(class_path, subclass_name)
        if not os.path.isdir(subclass_path):
            continue
        class_idx = class_to_idx.get(class_name)
        if class_idx is None:
            print(f"Предупреждение: неизвестный класс {class_name}")
            continue
        images = glob(os.path.join(subclass_path, '*.jpg')) + \
                 glob(os.path.join(subclass_path, '*.png')) + \
                 glob(os.path.join(subclass_path, '*.jpeg'))
        for img_path in images:
            all_images.append(img_path)
            all_labels.append(class_idx)

print(f"Всего изображений: {len(all_images)}")
print(f"Распределение классов: {dict(Counter(all_labels))}")

In [ ]:
# Веса для классов с учётом дисбаланса и усилением слабых классов
num_classes = len(class_to_idx)
class_weights = torch.zeros(num_classes)

train_counts_dict = dict(Counter(all_labels))
max_count = max(train_counts_dict.values()) if train_counts_dict else 1

# Базовые веса – обратная частота
for i in range(num_classes):
    count = train_counts_dict.get(i, 0)
    if count > 0:
        class_weights[i] = max_count / count
    else:
        class_weights[i] = 0.0

print("Raw class weights (inverse frequency):", class_weights.cpu().numpy())

# Нормировка, чтобы сумма равнялась num_classes
class_weights = class_weights / class_weights.sum() * num_classes

# Список слабых классов (скорректируйте под ваши наблюдения)
WEAK_CLASSES = [0, 2, 6, 8, 14]  # Апельсины, Груши, Киви, Лук, Яблоки красные
BOOST_FACTOR = 1.2  # во сколько раз увеличить вес для этих классов

for i in WEAK_CLASSES:
    class_weights[i] *= BOOST_FACTOR

# Повторная нормировка после усиления
class_weights = class_weights / class_weights.sum() * num_classes

print("Class weights after boosting weak classes:", class_weights.cpu().numpy())

# Перенесите эту строку ПОЗЖЕ, после определения device
# class_weights = class_weights.to(device)

# Вернёмся к обучению

### Аугментации

In [ ]:
def create_tta_batch(x: torch.Tensor):
    # x: shape (B, C, H, W)
    tta_versions = []

    tta_versions.append(x)
    tta_versions.append(torch.flip(x, dims=[3]))
    tta_versions.append(torch.flip(x, dims=[2]))
    tta_versions.append(torch.flip(x, dims=[2, 3]))
    tta_versions.append(torch.rot90(x, k=1, dims=[2, 3]))
    tta_versions.append(torch.rot90(x, k=2, dims=[2, 3]))
    tta_versions.append(torch.rot90(x, k=3, dims=[2, 3]))

    tta_batch = torch.cat(tta_versions, dim=0) # shape: (B*n_tta, C, H, W)
    return tta_batch, len(tta_versions)

def calc_tta_logits(model, X_batch):
    tta_batch, n_tta = create_tta_batch(X_batch) # shape: (B*n_tta, C, H, W)
    logits_all = model(tta_batch)  # shape: (B*n_tta, num_classes)

    B = X_batch.size(0)

    logits_all = logits_all.view(n_tta, B, -1) # shape: (n_tta, B, num_classes)
    logits = logits_all.mean(dim=0)  # shape: (B, num_classes)
    return logits


In [ ]:
def mixup_data(x, y, alpha=0.2):
    """
    Создаёт смешанные изображения и метки.
    x: батч изображений (B, C, H, W)
    y: батч меток (B,) или one-hot
    alpha: параметр бета-распределения
    Возвращает:
        mixed_x: смешанные изображения
        y_a, y_b: исходные метки для первого и второго изображений
        lam: коэффициент смешивания
    """
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(x.device)

    mixed_x = lam * x + (1 - lam) * x[index]
    y_a, y_b = y, y[index]

    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    """
    Критерий для MixUp: взвешенная сумма потерь для двух наборов меток.
    criterion: функция потерь, поддерживающая стандартный вызов (pred, target)
    pred: предсказания модели
    y_a, y_b: исходные метки
    lam: коэффициент смешивания
    """
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [ ]:
# ---------- НОВЫЙ БЛОК: несколько pipeline'ов аугментаций ----------
# Индексы проблемных классов (из class_to_idx)
WEAK_CLASSES = [0, 2, 6, 8, 14]       # Апельсины, Груши, Киви, Лук, Яблоки красные

IMAGE_W, IMAGE_H = 320, 320

# Базовый pipeline (для всех, кроме особых случаев)
base_train_transforms = A.Compose([
    A.Resize(IMAGE_W, IMAGE_H),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.3),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.15, rotate_limit=30, p=0.5),
    A.GridDropout(ratio=0.2, p=0.2),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    A.ToTensorV2(),
])

# Усиленный pipeline для слабых классов (больше геометрии, Cutout, шум)
strong_transforms = A.Compose([
    A.Resize(IMAGE_W, IMAGE_H),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=45, p=0.7),
    A.GridDropout(ratio=0.25, p=0.25),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    A.ToTensorV2(),
])

# Для валидации оставляем без изменений (только Resize + Normalize)
val_transforms = A.Compose([
    A.Resize(IMAGE_W, IMAGE_H),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    A.ToTensorV2(),
])

In [ ]:
@torch.no_grad()
def evaluate(model, dataloader, loss_fn, device, desc="Val"):
    model.eval()

    total_loss = 0.0
    total_correct = 0
    total_samples = 0

    pbar = tqdm(dataloader, desc=desc, leave=False)
    for X_batch, y_batch in pbar:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        logits = calc_tta_logits(model, X_batch)
        loss = loss_fn(logits, y_batch)

        batch_size = y_batch.size(0)
        total_loss += loss.item() * batch_size

        y_pred = logits.argmax(dim=1)
        total_correct += (y_pred == y_batch).sum().item()
        total_samples += batch_size

        avg_loss = total_loss / max(total_samples, 1)
        acc = total_correct / max(total_samples, 1)
        pbar.set_postfix(loss=f"{avg_loss:.4f}", acc=f"{acc:.4f}")

    avg_loss = total_loss / max(total_samples, 1)
    accuracy = total_correct / max(total_samples, 1)
    return accuracy, avg_loss

In [ ]:
def train(model, loss_fn, optimizer, scheduler, train_loader, val_loader, device, 
          ema=None, writer=None, n_epoch=10, save_path="best_model.pth"):
    num_iter = 0
    best_val_acc = 0.0
    patience = 5
    patience_counter = 0
    
    # Параметры MixUp
    mixup_alpha = 0.1
    mixup_prob = 0.5

    for epoch in range(1, n_epoch + 1):
        model.train()

        total_loss = 0.0
        clean_correct = 0
        clean_samples = 0
        num_batches = 0

        pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{n_epoch}", leave=True)

        for X_batch, y_batch in pbar:
            X_batch = X_batch.to(device, non_blocking=True)
            y_batch = y_batch.to(device, non_blocking=True)

            # Случайно решаем, применять ли MixUp
            use_mixup = random.random() < mixup_prob

            if use_mixup:
                X_batch, y_a, y_b, lam = mixup_data(X_batch, y_batch, alpha=mixup_alpha)
                logits = model(X_batch)
                loss = mixup_criterion(loss_fn, logits, y_a, y_b, lam)
                # Для батчей с MixUp не обновляем accuracy
            else:
                logits = model(X_batch)
                loss = loss_fn(logits, y_batch)
                
                # Считаем accuracy только на чистых батчах
                y_pred = logits.argmax(dim=1)
                clean_correct += (y_pred == y_batch).sum().item()
                clean_samples += y_batch.size(0)

            optimizer.zero_grad(set_to_none=True)
            loss.backward()
            optimizer.step()
            
            if ema is not None:
                ema.update(model)

            # Обновляем метрики
            batch_size = X_batch.size(0)
            total_loss += loss.item() * batch_size
            num_batches += 1
            
            avg_loss = total_loss / (num_batches * train_loader.batch_size)
            clean_acc = clean_correct / max(clean_samples, 1)
            
            pbar.set_postfix(
                train_loss=f"{avg_loss:.4f}", 
                train_acc=f"{clean_acc:.4f}",
                mixup=f"{use_mixup}"
            )

            # Логирование
            num_iter += 1
            if writer is not None:
                writer.add_scalar("Loss/train", loss.item(), num_iter)
                if not use_mixup:
                    writer.add_scalar("Accuracy/train_clean", (y_pred == y_batch).float().mean().item(), num_iter)

        # Валидация (используем EMA модель, если она есть)
        val_model = ema.ema if ema is not None else model
        val_acc, val_loss = evaluate(val_model, val_loader, loss_fn, device, desc=f"Val {epoch}/{n_epoch}")
        
        # Сохраняем лучшую модель
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
            torch.save(val_model.state_dict(), save_path)
            print(f"🔥 New best model saved! val_acc = {val_acc:.4f}")
        else:
            patience_counter += 1

        # Обновляем scheduler
        if scheduler is not None:
            if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                scheduler.step(val_loss)
            else:
                scheduler.step()
            current_lr = scheduler.get_last_lr()[0]
        else:
            current_lr = optimizer.param_groups[0]['lr']

        if writer is not None:
            writer.add_scalar("Loss/val", val_loss, num_iter)
            writer.add_scalar("Accuracy/val", val_acc, num_iter)
            writer.add_scalar("LR", current_lr, epoch)

        # Early stopping
        if patience_counter >= patience:
            print(f"⛔ Early stopping triggered. Best val_acc = {best_val_acc:.4f}")
            break

        print(f"Epoch {epoch}/{n_epoch}: val_loss={val_loss:.4f}  val_acc={val_acc:.4f}  lr={current_lr:.6f}")

    return model

In [ ]:
# ============================================
# НАЧАЛО K-FOLD ОБУЧЕНИЯ
# ============================================

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Переносим веса классов на устройство
class_weights = class_weights.to(device)

# Параметры K-Fold
n_folds = 5
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=SEED)
EPOCHS_PER_FOLD = 30  # можно увеличить до 50, если позволяет время
BATCH_SIZE = 32  # 

# Список для хранения путей к лучшим моделям
best_model_paths = []

# Цикл по фолдам
for fold, (train_idx, val_idx) in enumerate(skf.split(all_images, all_labels)):
    print(f"\n{'='*60}")
    print(f"Fold {fold+1}/{n_folds}")
    print(f"{'='*60}")
    
    # Формируем данные для текущего фолда
    train_data = [(all_images[i], all_labels[i]) for i in train_idx]
    val_data = [(all_images[i], all_labels[i]) for i in val_idx]
    
    print(f"Train samples: {len(train_data)}, Val samples: {len(val_data)}")
    
    # Создаём датасеты
    train_dataset_fold = MyDataset(
        images_filepaths=train_data,
        name2label=None,
        base_transform=base_train_transforms,
        strong_transform=strong_transforms,
        weak_classes=WEAK_CLASSES
    )
    
    val_dataset_fold = MyDataset(
        images_filepaths=val_data,
        name2label=None,
        base_transform=val_transforms,
        strong_transform=val_transforms
    )
    
    # Даталоадеры
    train_loader_fold = DataLoader(
        train_dataset_fold,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=2,
        pin_memory=True,
        persistent_workers=True
    )
    
    val_loader_fold = DataLoader(
        val_dataset_fold,
        batch_size=BATCH_SIZE // 2,
        shuffle=False,
        num_workers=2,
        pin_memory=True,
        persistent_workers=True
    )
    
    # Создаём модель для фолда
    model_fold = timm.create_model('tf_efficientnetv2_s', pretrained=True, num_classes=15, drop_rate=0.3)
    model_fold.to(device)
    
    # Loss, оптимизатор, scheduler, EMA
    loss_fn_fold = torch.nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.1)
    
    optimizer_fold = torch.optim.AdamW(model_fold.parameters(), lr=1e-3, weight_decay=1e-4)
    scheduler_fold = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer_fold, T_0=5, T_mult=1)
    
    ema_fold = EMA(model_fold, decay=0.999)
    
    # Путь для сохранения лучшей модели текущего фолда
    fold_save_path = f"best_model_fold{fold}.pth"
    best_model_paths.append(fold_save_path)
    
    # Обучение
    train(
        model=model_fold,
        loss_fn=loss_fn_fold,
        optimizer=optimizer_fold,
        scheduler=scheduler_fold,
        train_loader=train_loader_fold,
        val_loader=val_loader_fold,
        device=device,
        ema=ema_fold,
        writer=None,  # можно создать SummaryWriter для каждого фолда, если нужно
        n_epoch=EPOCHS_PER_FOLD,
        save_path=fold_save_path
    )
    
    # Очистка памяти
    del model_fold, train_loader_fold, val_loader_fold, train_dataset_fold, val_dataset_fold
    torch.cuda.empty_cache()
    
print("\n✅ Все фолды обучены! Лучшие модели сохранены.")
print("Пути к моделям:", best_model_paths)

# ============================================
# КОНЕЦ K-FOLD ОБУЧЕНИЯ
# ============================================

# Посмотрим на итоговые метрики

In [ ]:
import numpy as np
import torch
from sklearn.metrics import classification_report, confusion_matrix

@torch.no_grad()
def sklearn_report(model, dataloader, device, idx2class=None, digits=4):
    model.eval()

    y_true, y_pred = [], []

    for X_batch, y_batch in dataloader:
        X_batch = X_batch.to(device, non_blocking=True)

        logits = calc_tta_logits(model, X_batch)
        preds = logits.argmax(dim=1).cpu().numpy()

        y_pred.append(preds)
        y_true.append(y_batch.numpy())

    y_true = np.concatenate(y_true)
    y_pred = np.concatenate(y_pred)

    # names for report
    if idx2class is None:
        target_names = None
        labels = None
    else:
        labels = sorted(idx2class.keys())
        target_names = [idx2class[i] for i in labels]

    rep = classification_report(
        y_true, y_pred,
        labels=labels,
        target_names=target_names,
        digits=digits,
        zero_division=0
    )
    print(rep)

# Оформляем предсказание

In [ ]:
# ============================================
# ФИНАЛЬНОЕ ПРЕДСКАЗАНИЕ С АНСАМБЛЕМ МОДЕЛЕЙ
# ============================================
test_images_dir = base_path + "test_images"
submission_path = base_path + "sample_submission.csv"
output_path = "/kaggle/working/submission.csv"

print(f'test_images_dir({test_images_dir})')
print(f'submission_path({submission_path})')

submission = pd.read_csv(submission_path)
image_ids = submission["image_id"].tolist()

# Список для хранения логитов каждой модели
all_fold_logits = []

for fold, model_path in enumerate(best_model_paths):
    print(f"\n📦 Загрузка модели фолда {fold} из {model_path}")
    
    # Создаём модель с той же архитектурой
    model_fold = timm.create_model('tf_efficientnetv2_s', pretrained=False, num_classes=15)
    model_fold.load_state_dict(torch.load(model_path, map_location='cpu'))
    model_fold.eval().to(device)
    
    fold_logits = []
    test_batch_size = 32  # уменьшаем для TTA
    
    with torch.no_grad():
        for start_idx in tqdm(range(0, len(image_ids), test_batch_size), desc=f"Predicting fold {fold}"):
            batch_ids = image_ids[start_idx:start_idx + test_batch_size]
            images = []
            
            for image_id in batch_ids:
                image_path = os.path.join(test_images_dir, image_id)
                image = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_COLOR)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = val_transforms(image=image)["image"]
                images.append(image)
            
            X_batch = torch.stack(images).to(device, non_blocking=True)
            logits = calc_tta_logits(model_fold, X_batch)  # применяем TTA
            fold_logits.append(logits.cpu())
    
    fold_logits = torch.cat(fold_logits, dim=0)
    all_fold_logits.append(fold_logits)
    
    # Очистка памяти
    del model_fold
    torch.cuda.empty_cache()

print("\n🔄 Усреднение предсказаний по всем фолдам...")
mean_logits = torch.mean(torch.stack(all_fold_logits), dim=0)
pred_labels = mean_logits.argmax(dim=1).tolist()

submission["label"] = pred_labels
submission.to_csv(output_path, index=False)

print(f"✅ Сабмит сохранён в {output_path}")
print("\nПервые 5 строк предсказаний:")
submission.head()